# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
BUDGET = 1e10
PRICE = 450
POINTS = 200

In [3]:
df_0 = pd.read_csv("datasets/geo_data_0.csv")
df_1 = pd.read_csv("datasets/geo_data_1.csv")
df_2 = pd.read_csv("datasets/geo_data_2.csv")

In [4]:
df_0.info()
print("-"*40)
df_1.info()
print("-"*40)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 

В данных нет пропусков и всё описывается подходящими типами данных

In [5]:
print("средний объем запасов в первом регионе", df_0["product"].mean())
print("средний объем запасов во втором регионе", df_1["product"].mean())
print("средний объем запасов в третьем регионе", df_2["product"].mean())

средний объем запасов в первом регионе 92.50000000000001
средний объем запасов во втором регионе 68.82500000000002
средний объем запасов в третьем регионе 95.00000000000004


In [6]:
df_0.drop_duplicates(inplace=True)
df_1.drop_duplicates(inplace=True)
df_2.drop_duplicates(inplace=True)

### Разделение на выборки и масштабирование признаков

In [7]:
features_0 = df_0[["f0", "f1", "f2"]]
target_0 = df_0["product"]

features_1 = df_1[["f0", "f1", "f2"]]
target_1 = df_1["product"]

features_2 = df_2[["f0", "f1", "f2"]]
target_2 = df_2["product"]


features_train_0, features_valid_0, target_train_0, target_valid_0 = \
train_test_split(features_0, target_0, test_size=0.25, random_state=12345)

features_train_1, features_valid_1, target_train_1, target_valid_1 = \
train_test_split(features_1, target_1, test_size=0.25, random_state=12345)

features_train_2, features_valid_2, target_train_2, target_valid_2 = \
train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

scaler = StandardScaler()

features_train_0 = scaler.fit_transform(features_train_0)
features_valid_0 = scaler.transform(features_valid_0)

features_train_1 = scaler.fit_transform(features_train_1)
features_valid_1 = scaler.transform(features_valid_1)

features_train_2 = scaler.fit_transform(features_train_2)
features_valid_2 = scaler.transform(features_valid_2)

## Обучение и проверка модели

### Обучение моделей

Для каждого региона обучим свою модель

In [8]:
def find_mse(model, features, target):
    predicted = model.predict(features)
    mse = mean_squared_error(target, predicted)
    print("MSE", mse)
    print("RMSE", mse**0.5)
    print("-"*30)
    return 

model_0 = LinearRegression()
model_1 = LinearRegression()
model_2 = LinearRegression()

model_0.fit(features_train_0, target_train_0)
model_1.fit(features_train_1, target_train_1)
model_2.fit(features_train_2, target_train_2)

print("Первый регион")
find_mse(model_0, features_valid_0, target_valid_0)
print("Второй регион")
find_mse(model_1, features_valid_1, target_valid_1)
print("Третий регион")
find_mse(model_2, features_valid_2, target_valid_2)

Первый регион
MSE 1412.2129364399243
RMSE 37.5794217150813
------------------------------
Второй регион
MSE 0.7976263360391157
RMSE 0.893099286775617
------------------------------
Третий регион
MSE 1602.3775813236196
RMSE 40.02970873393434
------------------------------


In [25]:
#ячейка ревьюера 

def predict(df):
    features = df[["f0", "f1", "f2"]]
    target = df["product"]

    features_train, features_valid, target_train, target_valid = \
    train_test_split(features, target, test_size=0.25, random_state=12345)

    scaler = StandardScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predicted)**0.5    
    print("RMSE", rmse)
    print("Средний запас предсказанного сырья", predicted.mean())
    '''Возвращаем таргет и предсказания с соответсвующими друг другу индексами '''
    return target_valid.reset_index(drop = True), pd.Series(predicted) 

In [26]:
#ячейка ревьюера 

target_valid_0, predicted_0 = predict(df_0)

RMSE 37.5794217150813
Средний запас предсказанного сырья 92.59256778438035


In [27]:
#ячейка ревьюера 

target_valid_0, predicted_0

(0         10.038645
 1        114.551489
 2        132.603635
 3        169.072125
 4        122.325180
             ...    
 24995    170.116726
 24996     93.632175
 24997    127.352259
 24998     99.782700
 24999    177.821022
 Name: product, Length: 25000, dtype: float64,
 0         95.894952
 1         77.572583
 2         77.892640
 3         90.175134
 4         70.510088
             ...    
 24995    103.037104
 24996     85.403255
 24997     61.509833
 24998    118.180397
 24999    118.169392
 Length: 25000, dtype: float64)

### Сравнение с константной моделью

In [9]:
predicted_valid_const_0 = pd.Series(target_train_0.mean(), index = target_valid_0.index)
predicted_valid_const_1 = pd.Series(target_train_1.mean(), index = target_valid_1.index)
predicted_valid_const_2 = pd.Series(target_train_2.mean(), index = target_valid_2.index)

mse_0 = mean_squared_error(target_valid_0, predicted_valid_const_0)
mse_1 = mean_squared_error(target_valid_2, predicted_valid_const_1)
mse_2 = mean_squared_error(target_valid_2, predicted_valid_const_2)
print("Среднее")
print("RMSE_0 =", mse_0 ** 0.5)
print("RMSE_1 =", mse_1 ** 0.5)
print("RMSE_2 =", mse_2 ** 0.5)

Среднее
RMSE_0 = 44.289591053907365
RMSE_1 = 51.89905860601883
RMSE_2 = 44.90234968510566


### Вывод
Для первого и третьего региона модели показывают RMSE близкий к константной модели. Для второго региона модель получилась достаточно точная

## Подготовка к расчёту прибыли

In [10]:
min_product = ((BUDGET/POINTS)/PRICE)/1000
min_product  

111.11111111111111

In [11]:
predicted_0 = pd.Series(model_0.predict(features_valid_0))
predicted_1 = pd.Series(model_1.predict(features_valid_1))
predicted_2 = pd.Series(model_2.predict(features_valid_2))

Необходимые запасы для безубыточной разработки месторождений больше, чем средние запасы в во всех регионах. То есть если выбирать скважины наугад, то ничего хорошего не выйдет

In [12]:
predicted_valid_0 = model_0.predict(features_valid_0)
predicted_valid_1 = model_1.predict(features_valid_1)
predicted_valid_2 = model_2.predict(features_valid_2)

## Расчёт прибыли и рисков 

Техникой бутсртеп смоделируем выбор 500 случайных скважин в каждом регионе, выберем по предсказаниям модели 200 скважин с наибольшими запасами и посчитаем прибыль

In [13]:
def input(predict, target):
    sorted_predict = predict.sort_values(ascending=False)
    
    return target[sorted_predict.index][:POINTS].sum()*PRICE*1000 - BUDGET

In [17]:
def bootstrap(predicted, target):
    target = target.reset_index(drop=True)
    target_sample = target.sample(n=500, random_state=state, replace=True)
    predicted_sample = predicted[target_sample.index]
    return input(predicted_sample, target_sample)

state = np.random.RandomState(12345)

profit_0 = []
profit_1 = []
profit_2 = []
for i in range(1000):
    profit_0.append(bootstrap(pd.Series(predicted_valid_0), target_valid_0))
    profit_1.append(bootstrap(pd.Series(predicted_valid_1), target_valid_1))
    profit_2.append(bootstrap(pd.Series(predicted_valid_2), target_valid_2))
    
profit_0 = pd.Series(profit_0)
profit_1 = pd.Series(profit_1)
profit_2 = pd.Series(profit_2)

print("Средняя прибыть в первом регионе", profit_0.mean()*1e-9, "млрд")
print("Средняя прибыть во втором регионе", profit_1.mean()*1e-9, "млрд")
print("Средняя прибыть в третьем регионе", profit_2.mean()*1e-9, "млрд")

Средняя прибыть в первом регионе 0.4147362686144614 млрд
Средняя прибыть во втором регионе 0.5131721693440103 млрд
Средняя прибыть в третьем регионе 0.41019840127401797 млрд


In [18]:
#ячейка ревьюера
'Риск убытков = {:.2%} '.format((pd.Series([-2, -1, 4, 5])<0).mean())

'Риск убытков = 50.00% '

Рассмотрим 2.5% квантили для каждого региона, то есть посмотрим до какого значения лежат 2.5% самых низких возможных дохода для каждого регона.

In [19]:
print(profit_0.quantile(q=0.025)*450e-6, "млрд")
print(profit_1.quantile(q=0.025)*450e-6, "млрд")
print(profit_2.quantile(q=0.025)*450e-6, "млрд")

-55587.65240221593 млрд
38398.38580173855 млрд
-63190.677263433055 млрд


In [20]:
print('Риск убытков в первом регионе = {:.2%} '.format((pd.Series(profit_0)<0).mean()))
print('Риск убытков в первом регионе = {:.2%} '.format((pd.Series(profit_1)<0).mean()))
print('Риск убытков в первом регионе = {:.2%} '.format((pd.Series(profit_2)<0).mean()))

Риск убытков в первом регионе = 7.00% 
Риск убытков в первом регионе = 0.90% 
Риск убытков в первом регионе = 8.50% 


## Вывод

Было проведено предварительное исследование данных, которое показало, что во всех регионах средние запасы скважин ниже, чем необходимые запасы для безубыточной разработки. Были обучены Линейные модели для каждого региона, по предсказаниям которых из 500 случано выбраных скважин отбирались 200 с наибольшими предсказанными запасами. Были найдены средние значения возможной прибыли для каждого региона и значения, ниже которых с 2.5% вероятностью окажется доход. По этим данным был выбран наиболее прерспективный регион для разработки - второй, как регион с наибольшей средней возможной прибылью